In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
from time import time
import gc
print(xgb.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
sample_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

In [4]:
sample_submission.head()

In [5]:
params = {'objective': 'binary:logistic',
          'tree_method': 'gpu_hist',
          'predictor': 'gpu_predictor',
          'lambda': 0.04605965699636567,
          'alpha': 0.03842772979823298,
          'colsample_bytree': 0.3155289537736205,
          'subsample': 0.7947859156013958,
          'learning_rate': 0.001,
          'max_depth': 13,
          'min_child_weight': 295,
          'eval_metric': 'auc',
          'seed' : 19947
         }

In [6]:
columns = test.columns[1:]
columns

In [7]:
target = train['target'].values
train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof.shape

In [8]:
xgtest = xgb.DMatrix(test[columns].values) 

In [9]:
%%time
N_FOLDS = 5
start_time = time()
kf = StratifiedKFold(N_FOLDS, shuffle=True, random_state=2021)
i=1
for train_index, val_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[columns].values[train_index], train[columns].values[val_index]
        y_train, y_val = target[train_index], target[val_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=100000, evals=watchlist, 
                verbose_eval=5000, 
                early_stopping_rounds = 300)
        
        preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        train_oof[val_index] = ypred
        test_preds += preds/N_FOLDS
        print('Time elapsed:', time()-start_time)
        clf.save_model('xgb_gpu_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        

In [10]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

In [12]:
sample_submission.to_csv('subxgb_nov.csv')